In [3]:
import asyncio
import json
import json5
import os
import pathlib
from pprint import pprint
import requests

In [4]:
cwd = pathlib.Path().cwd()
proj_dir = cwd.parent.parent
os.chdir(proj_dir)

from utils.arcgis_helpers import checkout_token
from utils.arcgis_helpers import AsyncArcGISRequester

In [5]:
nifc_token = checkout_token('NIFC_AGO', 120, 'NIFC_TOKEN', 15)

In [ ]:
# must delete all records prior to updating fields 
# because some fields that are being removed and re-added to retain order are non-nullable

async with AsyncArcGISRequester() as requester:
    response = await requester.applyEdits_request(
        url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/AK_Wildfire_Values_at_Risk/FeatureServer/0',
        token=nifc_token,
        features_to_add=[],
        get_oids_to_delete_query='1=1'
    )

Entering async context...
Exiting async context...


In [25]:
service_0 = r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/admin/services/AK_Wildfire_Values_at_Risk/FeatureServer/0'

with open (r'C:\REPOS\con-j-e\ak-wildfire-values-at-risk-MERGING-TEMP\feature_service_management\layer_definition_updates\create_interior_feats_fields_ordered.json5', 'r') as file:
    update_fields = json5.load(file)

In [26]:
field_names = [field["name"] for field in update_fields["fields"]]

# in production service, _Interior fields were never added (so don't need to get deleted prior to updating)
# in production service, PertolTerm typo is still present (must delete this name explicitly instead of corrected version)
delete_fields = {
    "fields": [
        {"name": field} for field in field_names if field != "PetrolTerm_Nearest" and '_Interior' not in field
    ]
}
delete_fields["fields"].append({"name": "PertolTerm_Nearest"})

# Roads_Nearest is no longer in use
update_fields = {
    "fields": [
        obj for obj in update_fields["fields"] if obj["name"] != "Roads_Nearest"
    ]
}

In [27]:
pprint(delete_fields, indent=4)

{   'fields': [   {'name': 'AkMine_FeatureCount'},
                  {'name': 'AkMine_Active_AttrCount'},
                  {'name': 'AkMine_Note_AttrCount'},
                  {'name': 'AkMine_Type_AttrCount'},
                  {'name': 'AkMine_Nearest'},
                  {'name': 'AkPowerLine_TotalFeet'},
                  {'name': 'AkPowerLine_CondType_FeetSum'},
                  {'name': 'AkPowerLine_Voltage_FeetSum'},
                  {'name': 'AkPowerLine_Nearest'},
                  {'name': 'Aksd_FeatureCount'},
                  {'name': 'Aksd_MainStruct_AttrSum'},
                  {'name': 'Aksd_OtherStruct_AttrSum'},
                  {'name': 'Aksd_Protection_AttrCount'},
                  {'name': 'Aksd_SiteCat_AttrCount'},
                  {'name': 'Aksd_Nearest'},
                  {'name': 'ComsTwr_FeatureCount'},
                  {'name': 'ComsTwr_Type_AttrCount'},
                  {'name': 'ComsTwr_Nearest'},
                  {'name': 'Community_FeatureCount'

In [28]:
post_data = {
    'deleteFromDefinition': json.dumps(delete_fields),
    'f': 'json',
    'token': nifc_token
}

resp = requests.post(
    url = f'{service_0}/deleteFromDefinition',
    data = post_data
)

In [29]:
print(resp.json())

{'success': True}


In [30]:
pprint(update_fields, indent=4)

{   'fields': [   {   'alias': 'AkMine_FeatureCount',
                      'defaultValue': None,
                      'domain': None,
                      'editable': True,
                      'name': 'AkMine_FeatureCount',
                      'nullable': True,
                      'sqlType': 'sqlTypeOther',
                      'type': 'esriFieldTypeSmallInteger'},
                  {   'alias': 'AkMine_Active_AttrCount',
                      'defaultValue': None,
                      'domain': None,
                      'editable': True,
                      'length': 5000,
                      'name': 'AkMine_Active_AttrCount',
                      'nullable': True,
                      'sqlType': 'sqlTypeOther',
                      'type': 'esriFieldTypeString'},
                  {   'alias': 'AkMine_Note_AttrCount',
                      'defaultValue': None,
                      'domain': None,
                      'editable': True,
                      'len

In [31]:
post_data = {
    'addToDefinition': json.dumps(update_fields),
    'f': 'json',
    'token': nifc_token
}

resp = requests.post(
    url = f'{service_0}/addToDefinition',
    data = post_data
)

In [32]:
print(resp.json())

{'success': True}
